In [3]:
#課題3
# interval.py: 区間演算クラス
# R.T.Kneusel, "Numbers and Computers", Springer, 2015.
# Interval -> [left, right]
# rmode.py: 丸めモード変更
# [Windows&Linux]
#  https://rafaelbarreto.wordpress.com/2009/03/30/controlling-fpu-rounding-modes-with-python/
# [macOS]
#  https://stackoverflow.com/questions/16000574/tie-breaking-of-round-with-numpy

# OS種別
# https://stackoverflow.com/questions/1854/python-what-os-am-i-running-on
import platform

# DLL呼び出し
# from ctypes import cdll
from ctypes import *

# 丸めモード定義
global FE_TOZERO, FE_DOWNWARD, FE_UPWARD, FE_TONEAREST
# set_rmode(丸めモード), 丸めモード = get_rmode()
global set_rmode, get_rmode

# OS名
os_name = platform.system()

# Windows
if os_name == 'Windows':
    msvcrt = cdll.msvcrt
    MW_RC = 0x00000300

    def set_rmode(mode):
        msvcrt._controlfp_s(0, mode, MW_RC)

    FE_TOZERO = 0x00000300
    FE_DOWNWARD = 0x00000100
    FE_UPWARD = 0x00000200
    FE_TONEAREST = 0x00000000

    def get_rmode():
        current_rmode = c_uint()
        msvcrt._controlfp_s(byref(current_rmode), 0, 0)

        return current_rmode.value & MW_RC

# Linux
elif os_name == 'Linux':
    from ctypes.util import find_library
    libm = cdll.LoadLibrary(find_library('m'))
    set_rmode, get_rmode = libm.fesetround, libm.fegetround
    # x86
    FE_TOZERO = 0xc00
    FE_DOWNWARD = 0x400
    FE_UPWARD = 0x800
    FE_TONEAREST = 0
# macOS
elif os_name == "Darwin":
    from ctypes.util import find_library
    libc = cdll.LoadLibrary(find_library('c'))
    set_rmode, get_rmode = libc.fesetround, libc.fegetround
    # x86
    FE_TOZERO = 0xc00
    FE_DOWNWARD = 0x400
    FE_UPWARD = 0x800
    FE_TONEAREST = 0


# 丸めモード確認
def print_rmode():
    rmode = get_rmode()

    if rmode == FE_TOZERO:
        print('切り捨て')
    elif rmode == FE_DOWNWARD:
        print('-Infへの丸め')
    elif rmode == FE_UPWARD:
        print('+Infへの丸め')
    elif rmode == FE_TONEAREST:
        print('最近偶数値丸め')
    else:
        print('不明 ', rmode)

    return rmode


# -------------------------------------
# Copyright (c) 2021 Tomonori Kouya
# All rights reserved.
# -------------------------------------

import math


class Interval:

    # 開始前のデフォルト丸めモード
    default_rmode = get_rmode()

    # コンストラクタ
    def __init__(self, left, right=None):
        if right is None:
            self.left = left
            self.right = left
        else:
            self.left = left
            self.right = right

    # + : 加算
    def __add__(self, y):
        set_rmode(FE_DOWNWARD)
        left = self.left + y.left
        set_rmode(FE_UPWARD)
        right = self.right + y.right
        set_rmode(Interval.default_rmode)

        return Interval(left, right)

    # - : 減算
    def __sub__(self, y):
        set_rmode(FE_DOWNWARD)
        left = self.left - y.right
        set_rmode(FE_UPWARD)
        right = self.right - y.left
        set_rmode(Interval.default_rmode)

        return Interval(left, right)

    # - :マイナス
    def __neg__(self):
        left = -self.right
        right = -self.left

        return Interval(left, right)

    # * : 乗算
    def __mul__(self, y):
        if not isinstance(y, Interval):
            y = Interval(float(y))
        set_rmode(FE_DOWNWARD)
        left = min([
            self.left * y.left,
            self.right * y.left,
            self.left * y.right,
            self.right * y.right
        ])
        set_rmode(FE_UPWARD)
        right = max([
            self.left * y.left,
            self.right * y.left,
            self.left * y.right,
            self.right * y.right
        ])
        set_rmode(Interval.default_rmode)

        return Interval(left, right)

    __rmul__ = __mul__

    # recip(x) = 1/x: 逆数
    def recip(self):
        if (self.left < 0.0) and (self.right > 0.0):
            return Interval(-math.inf, math.inf)

        set_rmode(FE_DOWNWARD)
        left = 1.0 / self.right
        set_rmode(FE_UPWARD)
        right = 1.0 / self.left
        set_rmode(Interval.default_rmode)

        return Interval(left, right)

    # / : 除算
    def __truediv__(self, y):
        return self.__mul__(y.recip())

    # 絶対値
    def abs(self):
        return max([math.abs(self.left), math.abs(self.right)])

    # 平方根
    def sqrt(self):
        if (self.left < 0) or (self.right < 0):
            return Interval(-math.nan, -math.nan)

        return Interval(math.sqrt(self.left), math.sqrt(self.right))

    # ** : べき乗
    def __pow__(self, n):
        if (n % 2) == 1:
            set_rmode(FE_DOWNWARD)
            left = self.left ** n
            set_rmode(FE_UPWARD)
            right = self.right ** n
        elif self.left >= 0:
            set_rmode(FE_DOWNWARD)
            left = self.left ** n
            set_rmode(FE_UPWARD)
            right = self.right ** n
        elif self.right < 0:
            set_rmode(FE_DOWNWARD)
            left = self.right ** n
            set_rmode(FE_UPWARD)
            right = self.left ** n
        else:
            left = 0.0
            set_rmode(FE_UPWARD)
            right = self.left ** n
            temp = self.right ** n
            if temp > right:
                right = temp

        set_rmode(Interval.default_rmode)
        return Interval(left, right)

    # 出力用
    def __str__(self):
        return "[ %g, %g ]" % (self.left, self.right)

    __repr__ = __str__


# -------------------------------------
# Copyright (c) 2021 Tomonori Kouya
# All rights reserved.
# -------------------------------------

x=[Interval(0.7501)]

const4=Interval(float(4))
const1=Interval(float(1))
for i in range(0,100):
    x.append(const4*x[i]*(const1-x[i]))

print('    i, [         x[i].left         ,          x.[i].right         ]')
for i in range(0,101):
    if i%10==0:
        print(f'{i:5d},[{x[i].left:25.17e},{x[i].right:25.17e}]')

    i, [         x[i].left         ,          x.[i].right         ]
    0,[  7.50099999999999989e-01,  7.50099999999999989e-01]
   10,[  8.44495953582817260e-01,  8.44495953621622331e-01]
   20,[  1.42919379590904727e-01,  1.42960069692795372e-01]
   30,[ -1.50087739433940874e+03,  8.13706409017080773e+02]
   40,[                     -inf,                      inf]
   50,[                     -inf,                      inf]
   60,[                     -inf,                      inf]
   70,[                     -inf,                      inf]
   80,[                     -inf,                      inf]
   90,[                     -inf,                      inf]
  100,[                     -inf,                      inf]
